In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import torch
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 4090'

In [3]:
from datasets import load_dataset

prothom_alo_dataset = load_dataset("text", data_files="../datasets/Bangla Prothom Alo.txt", split="train")

Found cached dataset text (/home/ashahri1/.cache/huggingface/datasets/text/default-66e892579a4abe66/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


In [4]:
prothom_alo_dataset.set_format("pandas")

In [5]:
prothom_alo_df = prothom_alo_dataset[:]

In [6]:
(prothom_alo_df=='').sum()

text    1350000
dtype: int64

In [7]:
prothom_alo_dataset.reset_format()

In [8]:
prothom_alo_dataset

Dataset({
    features: ['text'],
    num_rows: 2700000
})

In [9]:
prothom_alo_dataset = prothom_alo_dataset.filter(lambda x: x["text"]!="")

Loading cached processed dataset at /home/ashahri1/.cache/huggingface/datasets/text/default-66e892579a4abe66/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2/cache-37bc92e3ecf68258.arrow


In [10]:
prothom_alo_dataset

Dataset({
    features: ['text'],
    num_rows: 1350000
})

In [11]:
# train_size = 10_000
# test_size = int(0.1 * train_size)

downsampled_dataset = prothom_alo_dataset.train_test_split(
    train_size=0.8, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1080000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 270000
    })
})

In [12]:
with open('../datasets/Bangla Error Words.txt', encoding='utf-8') as f:
    lines = f.readlines()

In [13]:
error_words = dict()

In [14]:
for line in lines:
    combination = line.split()
    original_word = combination[0]
    modified_words = combination[1:]
    error_words[original_word] = modified_words

In [15]:
error_words['গত']

['গইত']

In [16]:
import numpy as np
np.random.seed(42)

def replace_error_word(sentence, error_words):
    for error_word in error_words.keys():
        if error_word in sentence:
            #print(error_word)
            index = np.random.randint(len(error_words[error_word]))
            sentence = sentence.replace(error_word, error_words[error_word][index])
            break
    return sentence

In [17]:
# np.random.randint(1,3)

In [18]:
# replace_error_word("তখন আমাদের দেখা হবে।", error_words)

In [19]:
# count = 0
# index = 0
# indices = list()
# for sample in downsampled_dataset["test"].select(range(10)):
#     #replace 15% of time
#     if np.random.random() < 0.15:
#         replaced_sample = replace_error_word(sample['text'], error_words)
#         sample['text'] = replaced_sample
#         print(sample['text'])
#         indices.append(index)
#         count += 1
#     print(index)
#     index += 1
        
# print(count)

In [20]:
def corrupt_text(examples):
    # Create a corrupt example
    #replace 15% of time
    if np.random.random() < 0.10:
        examples["text"] = replace_error_word(examples['text'], error_words)
    return examples

In [21]:
downsampled_dataset["test"] = downsampled_dataset["test"].map(corrupt_text)

Map:   0%|          | 0/270000 [00:00<?, ? examples/s]

In [22]:
downsampled_dataset["test"][:10]

{'text': ['যত দ্রুত সম্ভব এই ঘুষ দুর্নীতি দমন করতে হবে ।',
  'ভর্তি পরীক্ষা স্থগিতের পেছনে এটিও একটি বড় কারণ ।',
  'বাংলা একাডেমির উদ্যোগে তাঁর 1111111111 গান ইংরেজিতে অনূদিত হয়েছে ।',
  'একই সময়ের মধ্যে বৌদ্ধ সম্প্রদায়ের জনসংখ্যার হেরফের হবে না ।',
  'ইউপি সদস্যরা ভয়ে আছেন ।',
  'তারা এই উদ্যোগকে দেশবিরোধিতার সঙ্গে তুলনা করেছে ।',
  'এর মাধ্যমে দেশের জকোনো প্রান্ত থেকে জকোনো ব্যক্তি অভিযোগ বা পরামর্শ দিতে পারবেন ।',
  'মৃত্যুদণ্ডের রায় শোনার পর নূর হোসেন ও র\u200d্যাবের সাবেক তিন কর্মকর্তা স্বাভাবিক ছিলেন ।',
  'এর জন্য জনমত গঠন করতে হবে ।',
  'খুব কমসংখ্যক বলা যায় নামমাত্র দু একজন সুযোগ পেয়ে যেতেন ।']}

In [23]:
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast(
    #tokenizer_object=tokenizer,
    tokenizer_file="unigram_tokenizer_prothom_alo.json", # You can load from the tokenizer file, alternatively
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
    return_special_tokens_mask = True,
    model_max_length = 512,
)

In [24]:
from transformers import BertConfig, BertForMaskedLM

unigram_bert_config = BertConfig.from_pretrained("configs/unigram_bert_config/config.json")
# Building the model from the config
# Model is randomly initialized
model = BertForMaskedLM(unigram_bert_config)

In [25]:
unigram_bert_config

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [26]:
def tokenize_function(examples):
    result = tokenizer(examples["text"], padding="max_length", max_length=80, truncation=True)
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

In [27]:
# Use batched=True to activate fast multithreading!
tokenized_datasets = downsampled_dataset.map(
    tokenize_function, batched=True, remove_columns=["text"]
)
tokenized_datasets

Map:   0%|          | 0/1080000 [00:00<?, ? examples/s]

Map:   0%|          | 0/270000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 1080000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 270000
    })
})

In [28]:
tokenized_datasets.remove_columns("token_type_ids")

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 1080000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 270000
    })
})

In [29]:
# temp = tokenized_datasets.filter(lambda x:x if 1 in x["input_ids"] else None)

In [30]:
# temp

In [31]:
tokenized_datasets['train'][0]

{'input_ids': [2,
  2528,
  826,
  12,
  1860,
  16,
  5,
  6,
  3,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

In [32]:
def group_texts(examples):
    # Create a new labels column
    examples["labels"] = examples["input_ids"].copy()
    return examples

In [33]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Map:   0%|          | 0/1080000 [00:00<?, ? examples/s]

Map:   0%|          | 0/270000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1080000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 270000
    })
})

In [34]:
downsampled_dataset["test"][:100]

{'text': ['যত দ্রুত সম্ভব এই ঘুষ দুর্নীতি দমন করতে হবে ।',
  'ভর্তি পরীক্ষা স্থগিতের পেছনে এটিও একটি বড় কারণ ।',
  'বাংলা একাডেমির উদ্যোগে তাঁর 1111111111 গান ইংরেজিতে অনূদিত হয়েছে ।',
  'একই সময়ের মধ্যে বৌদ্ধ সম্প্রদায়ের জনসংখ্যার হেরফের হবে না ।',
  'ইউপি সদস্যরা ভয়ে আছেন ।',
  'তারা এই উদ্যোগকে দেশবিরোধিতার সঙ্গে তুলনা করেছে ।',
  'এর মাধ্যমে দেশের জকোনো প্রান্ত থেকে জকোনো ব্যক্তি অভিযোগ বা পরামর্শ দিতে পারবেন ।',
  'মৃত্যুদণ্ডের রায় শোনার পর নূর হোসেন ও র\u200d্যাবের সাবেক তিন কর্মকর্তা স্বাভাবিক ছিলেন ।',
  'এর জন্য জনমত গঠন করতে হবে ।',
  'খুব কমসংখ্যক বলা যায় নামমাত্র দু একজন সুযোগ পেয়ে যেতেন ।',
  'আজ কারও কাছ থাকে প্রেমের প্রস্তাব পেতে পারেন ।',
  'তাই এই কালেকশনের জন্য এ দুজন ছাড়া আর কোনো বিকল্প ছিল না ।',
  'গতকাল সোমবার সন্ধ্যায় নগরীর চাষাঢ়া কেন্দ্রীয় শহীদ মিনারে এই কর্মসূচি পালন করা হয় ।',
  'প্রতিটি দল এ সময়ে ছয়টি সিরিজ খেলবে তিনটি ঘরের মাঠে ও তিনটি বাইরে ।',
  'সাংঘাতিক ব্যক্তিত্বসম্পন্ন ও মেধাবী একটা মেয়ে ।',
  'তখন দুই ধরনের খবর ছড়িয়ে পড়ায় দেখা দেয় বিভ্রান্তি ।',
  'প

In [37]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'[CLS] শীত মৌসুমে যেকোনো জায়গায় বসা যায় ।[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]'

In [38]:
tokenizer.decode(lm_datasets["train"][1]["labels"])

'[CLS] শীত মৌসুমে যেকোনো জায়গায় বসা যায় ।[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]'

In [39]:
tokenizer.pad_token_id

1

In [40]:
import collections
import numpy as np
np.random
from transformers import default_data_collator

wwm_probability = 0.15


def bangla_data_collator(features):
    for feature in features:
#         word_ids = feature.pop("word_ids")

#         # Create a map between words and corresponding token indices
#         mapping = collections.defaultdict(list)
#         current_word_index = -1
#         current_word = None
#         for idx, word_id in enumerate(word_ids):
#             if word_id is not None:
#                 if word_id != current_word:
#                     current_word = word_id
#                     current_word_index += 1
#                 mapping[current_word_index].append(idx)

        # Randomly mask words
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        mask = np.random.binomial(1, wwm_probability, (len(input_ids),))
        special_tokens =  [tokenizer.unk_token_id, tokenizer.pad_token_id, tokenizer.cls_token_id, \
                           tokenizer.sep_token_id, tokenizer.mask_token_id]
        
        new_labels = [-100] * len(labels)
        for idx in np.where(mask)[0]:
#             word_id = word_id.item()
#             print(word_id)
#             for idx in mapping[word_id]:
#             if word_ids[idx] is not None:
            if input_ids[idx] not in special_tokens:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
            feature["labels"] = new_labels
        
    return default_data_collator(features)

In [41]:
from transformers import DataCollatorForLanguageModeling

# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
lm_datasets = lm_datasets.remove_columns(["word_ids"])
data_collator = bangla_data_collator

In [42]:
samples = [lm_datasets["train"][i] for i in range(3)]
# for sample in samples:
#     _ = sample.pop("word_ids")

for chunk in bangla_data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] হাওরে একবারই ফসল হয়[MASK]।[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]'

'>>> [CLS][MASK] মৌসুমে যেকোনো জায়গায় বসা যায় ।[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]'

'>>> [CLS] সৌম্যর ব্যাটিং সব সময়ই দৃষ্টিসুখকর হয় ।[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PA

In [43]:
samples = [lm_datasets["train"][i] for i in range(1)]

chunk = data_collator(samples)
print(chunk["input_ids"])
print(chunk["labels"])

tensor([[   2, 2528,    4,   12, 1860,    4,    5,    4,    3,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1]])
tensor([[-100, -100,  826, -100, -100,   16, -100,    6, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100,

In [44]:
model = BertForMaskedLM.from_pretrained("models/unigram/bert-base-pretrained-prothom-alo")

In [45]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("models/unigram/bert-base-pretrained-prothom-alo")

In [46]:
#  disable weights and biases logging
import os
os.environ["WANDB_DISABLED"] = "true"

In [47]:
from transformers import TrainingArguments

batch_size = 64
eval_batch_size = 64
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size


training_args = TrainingArguments(
    num_train_epochs = 3,
    report_to = None,
    output_dir="models/unigram/bert-base-pretrained-prothom-alo",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=eval_batch_size,
    #push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps,
    load_best_model_at_end=True,
    save_strategy = "epoch",
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [48]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Using amp half precision backend


In [49]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 270000
  Batch size = 64


>>> Perplexity: 29.27


In [64]:
# text = 'পরে সেখানে সংক্ষিপ্ত সমাবেশ অনুষ্ঠিত হয় ।'
text =  '[MASK] সেখানে সংক্ষিপ্ত সমাবেশ অনুষ্ঠিত হয় ।'

In [65]:
tokenizer.tokenize(text)

['[MASK]', '▁সেখানে', '▁সংক্ষিপ্ত', '▁সমাবেশ', '▁অনুষ্ঠিত', '▁হয়', '▁', '।']

In [66]:
import torch 

inputs = tokenizer(text, return_tensors="pt")
inputs.to("cuda")

token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> পরে সেখানে সংক্ষিপ্ত সমাবেশ অনুষ্ঠিত হয় ।'
'>>> এরপর সেখানে সংক্ষিপ্ত সমাবেশ অনুষ্ঠিত হয় ।'
'>>> বিকেলে সেখানে সংক্ষিপ্ত সমাবেশ অনুষ্ঠিত হয় ।'
'>>> সেখানে সেখানে সংক্ষিপ্ত সমাবেশ অনুষ্ঠিত হয় ।'
'>>> দুপুরে সেখানে সংক্ষিপ্ত সমাবেশ অনুষ্ঠিত হয় ।'
